In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

np.set_printoptions(threshold=np.inf)
def normalize(df):
    result = df.copy()
    for column in df.columns:
        max_value = df[column].max()
        min_value = df[column].min()
        if(max_value-min_value!=0):
            result[column] = (df[column] - min_value) / (max_value - min_value) 
        else:
            result[column]=df[column]
    return(result)  
normalised_train=normalize(pd.read_csv("./medical_training_data.csv"))
normalised_test=normalize(pd.read_csv("./medical_testing_data.csv"))


normalised_train_data=np.transpose(normalised_train.iloc[:,1:16].to_numpy())


train_result=normalised_train.iloc[:,16].to_numpy()
train_result=np.transpose(train_result[:,np.newaxis])

normalised_test_data=np.transpose(normalised_test.iloc[:,1:16].to_numpy())

test_result=normalised_test.iloc[:,16].to_numpy()
test_result=np.transpose(test_result[:,np.newaxis])


In [29]:
import numpy as np


class ann:
    def __init__(self,train,train_result,test,test_result,epoches,alpha,hidden_layers,l):  # l to input the activation functions between each layer  #  hidden layers is a list passed in which number of features for each hidden layer 
        
        self.train=train
        self.train_result=train_result
        self.test=test 
        self.test_result=test_result
        self.epoches=epoches
        self.alpha=alpha
        self.l=l          # activation function between each rows

        self.input_features=np.shape(train)[0] # along the rows 
        self.output_features = np.shape(train_result)[0]
        self.m=np.shape(train)[1]      # along the columns in a row    

        self.layers=[self.input_features]+hidden_layers+[self.output_features]   # contains number of features in each layer including start , hidden and end layers


        weights=[]     
        bias=[]
         
        
        error_wrt_z=[]
        error_wrt_w=[]
        error_wrt_b=[]

        for i in range(len(self.layers)-1): 
            a=np.random.rand (self.layers[i+1],self.layers[i])
            weights.append(a)

            b=np.random.rand(self.layers[i+1],1)          # will use broad casting                                              
            bias.append(b) 

            error_wrt_w.append(i)    # how many dw and db will be there , later i's will be updated with dw and db matrices 

            error_wrt_b.append(i)      # there will be number of (layers - 1) matrices for weights,bias,error wrt w,b,z in each forward propagation   

            error_wrt_z.append(i)  
        
        self.weights=weights
        self.bias=bias

       
        self.error_wrt_z= error_wrt_z
        self.error_wrt_w= error_wrt_w
        self.error_wrt_b= error_wrt_b

        self.activations=[]
        self.z=[]

        self.function_derivitive=[self.reluDerivative,self.sigmoid_derivative]  # in middle layers put relu or sigmoid only you cannnot put softmax in middle as you dont know d softmax(z) / d(z)
        self.function=[self.relu,self.sigmoid,self.softmax]  

    

    def relu(self,x):
        return np.maximum(0.0, x)


    def sigmoid(self , x):
        y=1/(1+np.exp(-x) )                       
        return y
        
    def sigmoid_derivative(self, x):        #  d sigma(z) / d(z) = sigma(z)*(1-sigma(z)) 
        return np.multiply(x,(1-x))         # in this we are finding for all the a's of the matrix 

    def reluDerivative(self,x):
        y=np.copy(x)
        y[y<=0] = 0
        y[y>0] = 1
        return y     

    def softmax (self,x):
        e=np.exp(x)
        return e/(np.sum(e,axis=0))  


    def forward_propogate(self,data): # we are using forward prop on training as well as testing data so 
        self.activations.clear()   #  with each epoch our w will be updated as dw db dz will be updated so we will be getting new activations as well as z  after each eppoch
        self.z.clear()
        self.z.append(data)
        self.activations.append(data)        ### note z and a will contain all the layers from  1st given layer at indeex 0 to the last output predicted layer 
      
        for i in range (len(self.layers)-1):
            self.z.append(np.add (np.dot(self.weights[i],self.activations[i]) ,self.bias[i]))
        
           # self.z.append(np.add( np.dot( self.weights[i],self.activations[i] ) ,np.repeat(self.bias[i],self.m,axis=1)) )    
            self.activations.append(self.function[self.l[i]](self.z[i+1]  ) )  



    def backward_propogation(self):  # we dont know d softmax(z) / d (z) so we are not using dz formula for last layer 
        self.error_wrt_z[len(self.layers)-2]=np.subtract(self.activations[len(self.layers)-1] , self.train_result )
        for i in reversed(range(len(self.layers) - 1)):
            self.error_wrt_w[i] = 1/self.m*(np.dot((self.error_wrt_z[i]),np.transpose(self.activations[i])))  
            
            self.error_wrt_b[i]=1/self.m *(np.sum(self.error_wrt_z[i] , axis=1 , keepdims=True))

            if i>=1:
                    self.error_wrt_z[i-1]=np.multiply (  np.dot ( np.transpose (self.weights[i]) , self.error_wrt_z[i] )  , self.function_derivitive[ self.l[i-1] ] ( self.function[ self.l[i-1] ]   (self.z[i]) ) )
            else:
                continue

    def training(self):
        for i in range(self.epoches):
            self.forward_propogate(data=self.train)
           
            final_prediction=self.predict()
            accuracy_percentage=self.accuracy(final_prediction,actual_result=self.train_result)
            print("Accuracy in epoch {} is {} percentage".format(i,accuracy_percentage))
            self.backward_propogation()
            for j in range(len (self.layers)-1):
                self.weights[j]=np.subtract(self.weights[j],self.alpha*self.error_wrt_w[j])
                
                self.bias[j]=np.subtract(self.bias[j],self.alpha*self.error_wrt_b[j])
    
    def predict(self):  
             # output is single (binary classigication using  sigmoid)   # [1 0 1 1 0 0 ] like this
       final_prediction=0.5<=self.activations[len(self.layers)-1] 
       return(final_prediction.astype(int))    
           
       
   
    def accuracy(self,final_prediction,actual_result):
        accuracy_per = (np.sum(actual_result ==final_prediction) / (actual_result).shape[1] ) * 100
        return accuracy_per
    # after training when we are having updated w  and b we predict result of testing set 
    
    def Testing(self):
        self.forward_propogate(data=self.test)  # we will get all activations layers including the last predicted layer 
        binary_prediction=self.predict()
        accuracy_percentage=self.accuracy(binary_prediction,actual_result=self.test_result)
        print("accuracy percentage on your  testing data is {} ".format(accuracy_percentage))


a=ann(train=normalised_train_data,train_result=train_result,test=normalised_test_data,test_result=test_result,epoches=250,alpha=0.001,hidden_layers=[8,4],l=[0,0,1])


a.training()
a.Testing()




Accuracy in epoch 0 is 15.643224699828473 percentage
Accuracy in epoch 1 is 15.643224699828473 percentage
Accuracy in epoch 2 is 15.643224699828473 percentage
Accuracy in epoch 3 is 15.643224699828473 percentage
Accuracy in epoch 4 is 15.643224699828473 percentage
Accuracy in epoch 5 is 15.643224699828473 percentage
Accuracy in epoch 6 is 15.643224699828473 percentage
Accuracy in epoch 7 is 75.64322469982847 percentage
Accuracy in epoch 8 is 83.9794168096055 percentage
Accuracy in epoch 9 is 84.28816466552315 percentage
Accuracy in epoch 10 is 84.35677530017152 percentage
Accuracy in epoch 11 is 84.35677530017152 percentage
Accuracy in epoch 12 is 84.35677530017152 percentage
Accuracy in epoch 13 is 84.35677530017152 percentage
Accuracy in epoch 14 is 84.35677530017152 percentage
Accuracy in epoch 15 is 84.35677530017152 percentage
Accuracy in epoch 16 is 84.35677530017152 percentage
Accuracy in epoch 17 is 84.35677530017152 percentage
Accuracy in epoch 18 is 84.35677530017152 percenta